In [9]:
import os
import pandas as pd
import glob
import matplotlib.pyplot as plt


In [10]:
# 데이터 읽기 및 NaN 값 처리
val = pd.read_csv('validation/val_combine.csv', low_memory=False)
# 데이터 읽기 및 NaN 값 처리
test = pd.read_csv('test/test_combine.csv', low_memory=False)
test

,datetime,timestamp,subject_id,x_last,x_mean,x_max,x_min,x_median,y_last,y_mean,...,heart_rate_min,heart_rate_median,w_light,burned_calories,distance,running_steps,speed,steps,step_frequency,walking_steps
0,2023-11-05 00:00:00,2023-11-05 00:00:00,5.0,-0.074220,-0.079756,0.088585,-0.462081,-0.079009,0.136469,0.108708,...,NaN,NaN,NaN,0.0,7.73,0.0,0.128833,10.0,0.166667,0.0
1,2023-11-05 00:01:00,2023-11-05 00:01:00,5.0,-0.071826,-0.078615,0.177171,-0.356736,-0.079009,0.105345,0.107999,...,NaN,NaN,NaN,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0
2,2023-11-05 00:02:00,2023-11-05 00:02:00,5.0,-0.079009,-0.078439,3.536236,-3.888184,-0.079009,0.114922,0.111475,...,NaN,NaN,NaN,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0
3,2023-11-05 00:03:00,2023-11-05 00:03:00,5.0,-0.071826,-0.068377,0.280122,-0.316035,-0.069432,0.095768,0.113084,...,NaN,NaN,NaN,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0
4,2023-11-05 00:04:00,2023-11-05 00:04:00,5.0,-0.040701,-0.070256,0.280122,-0.720655,-0.071826,0.112527,0.113228,...,NaN,NaN,NaN,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174225,2023-11-09 23:55:00,2023-11-09 23:55:00,8.0,0.069434,0.063908,0.138868,0.016760,0.064645,0.263370,0.255261,...,0.0,0.0,17.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0
174226,2023-11-09 23:56:00,2023-11-09 23:56:00,8.0,0.057463,0.062560,0.098165,0.023943,0.062251,0.258582,0.255150,...,0.0,0.0,17.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0
174227,2023-11-09 23:57:00,2023-11-09 23:57:00,8.0,0.067040,0.062821,0.090982,0.028731,0.062251,0.251399,0.255240,...,0.0,0.0,17.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0
174228,2023-11-09 23:58:00,2023-11-09 23:58:00,8.0,0.050280,0.061853,0.093377,0.028731,0.062251,0.265765,0.256747,...,0.0,39.0,17.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0


In [11]:
import math

# 함수 정의
def calculate_expression(x, y, z):
    numerator = z ** 2
    denominator = math.sqrt(x ** 2 + y ** 2)
    if denominator == 0:
        return 0  # 분모가 0인 경우 각도를 0으로 설정
    angle_radians = math.atan(numerator / denominator)
    angle_degrees = angle_radians * 180 / math.pi
    return angle_degrees

# 각 행에 대해 계산을 수행하고 새로운 컬럼을 추가
val['z_angle'] = val.apply(lambda row: calculate_expression(row['x_median'], row['y_median'], row['z_median']), axis=1)
test['z_angle'] = test.apply(lambda row: calculate_expression(row['x_median'], row['y_median'], row['z_median']), axis=1)

In [12]:
# 함수 정의
def calculate_expression(x, y, z):
    result = math.sqrt(x**2 + y**2 + z**2) - 1
    return max(0, result)

# 새로운 데이터프레임 생성 및 계산 적용
val['ENMO'] = val.apply(lambda row: calculate_expression(row['x_median'], row['y_median'], row['z_median']), axis=1)
test['ENMO'] = test.apply(lambda row: calculate_expression(row['x_median'], row['y_median'], row['z_median']), axis=1)

In [13]:
import re

# Function to extract the first label
def extract_first_label(text):
    if isinstance(text, str):
        match = re.search(r"\[\s*'([^']+)'", text)
        return match.group(1) if match else None
    return None

# Apply the function to the column
val['primary_ambience'] = val['ambience_labels'].apply(extract_first_label)
test['primary_ambience'] = test['ambience_labels'].apply(extract_first_label)

# Print the result
print(val[['subject_id', 'timestamp', 'primary_ambience']])

        subject_id            timestamp primary_ambience
0              1.0  2023-08-20 00:00:00           Speech
1              1.0  2023-08-20 00:01:00           Speech
2              1.0  2023-08-20 00:02:00            Music
3              1.0  2023-08-20 00:03:00            Music
4              1.0  2023-08-20 00:04:00           Speech
...            ...                  ...              ...
171353         4.0  2023-11-01 23:53:00          Vehicle
171354         4.0  2023-11-01 23:54:00          Vehicle
171355         4.0  2023-11-01 23:55:00          Vehicle
171356         4.0  2023-11-01 23:56:00          Vehicle
171357         4.0  2023-11-01 23:57:00          Vehicle

[171358 rows x 3 columns]


In [14]:
list(val['m_usage_stats'].unique())

['[]',
 "[{'app_name': 'One UI 홈', 'total_time': 6886}]",
 "[{'app_name': 'Google Play 스토어', 'total_time': 13491}\n {'app_name': '게임런처', 'total_time': 397}\n {'app_name': '탕탕특공대', 'total_time': 210550}\n {'app_name': 'One UI 홈', 'total_time': 10039}]",
 "[{'app_name': 'Google Play 스토어', 'total_time': 17667}\n {'app_name': '게임런처', 'total_time': 397}\n {'app_name': '탕탕특공대', 'total_time': 315900}\n {'app_name': 'One UI 홈', 'total_time': 3153}]",
 "[{'app_name': 'Google Play 스토어', 'total_time': 17667}\n {'app_name': '게임런처', 'total_time': 397}\n {'app_name': '탕탕특공대', 'total_time': 321601}\n {'app_name': 'One UI 홈', 'total_time': 3153}]",
 "[{'app_name': 'Google Play 스토어', 'total_time': 4176}\n {'app_name': '게임런처', 'total_time': 215}\n {'app_name': '탕탕특공대', 'total_time': 111051}]",
 "[{'app_name': '게임런처', 'total_time': 215}\n {'app_name': '탕탕특공대', 'total_time': 5701}]",
 "[{'app_name': '게임런처', 'total_time': 215}]",
 "[{'app_name': '게임런처', 'total_time': 379}\n {'app_name': 'One UI 홈', 'total_

In [16]:
# m_usage_stats 열을 드롭
val = val.drop(columns=['m_usage_stats'])
test = test.drop(columns=['m_usage_stats'])

In [17]:
val['primary_ambience'].unique()

array(['Speech', 'Music', 'Inside, small room', 'Mechanical fan',
       'Mechanisms', 'Camera', 'Beep, bleep', 'Spray', 'Explosion',
       'Jackhammer', 'White noise', 'Insect', 'Snake', 'Breathing',
       'Animal', 'Vehicle', 'Burping, eructation', 'Engine starting',
       'Vehicle horn, car horn, honking', 'Cacophony', 'Radio', 'Engine',
       'Sound effect', 'Silence', 'Snoring', 'Hum', 'Noise', 'Mains hum',
       'Electronic tuner', 'Dishes, pots, and pans', 'Hammer', 'Writing',
       'Water tap, faucet', 'Fireworks', 'Synthesizer', 'Crackle',
       'Clip-clop', 'Skateboard', 'Wind', 'Helicopter',
       'Outside, rural or natural', 'Wood', 'Glass', 'Bee, wasp, etc.',
       'Water', 'Toothbrush', 'Toilet flush', 'Chink, clink',
       'Outside, urban or manmade', 'Hiss', 'Jingle, tinkle',
       'Filing (rasp)', 'Groan', 'Gunshot, gunfire', 'Gears',
       'Television', 'Rail transport', 'Bathtub (filling or washing)',
       'Traffic noise, roadway noise', 'Burst, pop', '

In [18]:
# 잠 관련 소리 목록
sleep_words = ['White noise', 'Silence', 'Snoring', 'Hum', 'Breathing', 'Rain', 'Lullaby', 'Heartbeat', 'Pink noise', 'Humming', 'Inside, small room']

def filter_sleep(word):
    if pd.isna(word):
        return 1
    elif any(word.lower() == w.lower() for w in sleep_words):
        return 1
    else:
        return 0
    
val['sleep_sound'] = val['primary_ambience'].apply(filter_sleep)
test['sleep_sound'] = test['primary_ambience'].apply(filter_sleep)

# 결과 확인
print(val[['datetime', 'primary_ambience', 'sleep_sound']].head())
print(test[['datetime', 'primary_ambience', 'sleep_sound']].head())

              datetime primary_ambience  sleep_sound
0  2023-08-20 00:00:00           Speech            0
1  2023-08-20 00:01:00           Speech            0
2  2023-08-20 00:02:00            Music            0
3  2023-08-20 00:03:00            Music            0
4  2023-08-20 00:04:00           Speech            0
              datetime primary_ambience  sleep_sound
0  2023-11-05 00:00:00           Speech            0
1  2023-11-05 00:01:00           Speech            0
2  2023-11-05 00:02:00           Speech            0
3  2023-11-05 00:03:00           Speech            0
4  2023-11-05 00:04:00           Speech            0


In [19]:
val.keys()

Index(['datetime', 'timestamp', 'subject_id', 'x_last', 'x_mean', 'x_max',
       'x_min', 'x_median', 'y_last', 'y_mean', 'y_max', 'y_min', 'y_median',
       'z_last', 'z_mean', 'z_max', 'z_min', 'z_median', 'm_activity',
       'ambience_labels', 'altitude_last', 'altitude_mean', 'altitude_max',
       'altitude_min', 'altitude_median', 'latitude_last', 'latitude_mean',
       'latitude_max', 'latitude_min', 'latitude_median', 'longitude_last',
       'longitude_mean', 'longitude_max', 'longitude_min', 'longitude_median',
       'speed_last', 'speed_mean', 'speed_max', 'speed_min', 'speed_median',
       'm_light', 'heart_rate_last', 'heart_rate_mean', 'heart_rate_max',
       'heart_rate_min', 'heart_rate_median', 'w_light', 'burned_calories',
       'distance', 'running_steps', 'speed', 'steps', 'step_frequency',
       'walking_steps', 'z_angle', 'ENMO', 'primary_ambience', 'sleep_sound'],
      dtype='object')

In [20]:
val['datetime'] = pd.to_datetime(val['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])
val['day'] = pd.to_datetime(val['datetime'].dt.date)
test['day'] = pd.to_datetime(test['datetime'].dt.date)

In [21]:
# GPS 엔트로피 계산
from scipy.stats import entropy

# 1. 그리드 셀로 공간을 나누기 위한 함수
def get_grid_cell(lat, lon, grid_size=0.01):
    lat_cell = int(lat / grid_size)
    lon_cell = int(lon / grid_size)
    return (lat_cell, lon_cell)

# 2. GPS 엔트로피 계산 함수 정의
def calculate_gps_entropy(df):
    # 결측치 제거한 데이터만 진행
    df_gps = df.dropna(subset=['latitude_median', 'longitude_median'])
    
    # 각 위치를 그리드 셀로 변환
    df_gps.loc[:, 'grid_cell'] = df_gps.apply(lambda row: get_grid_cell(row['latitude_median'], row['longitude_median']), axis=1)
    
    # 사용자와 날짜별로 그룹화
    grouped = df_gps.groupby(['subject_id', 'day'])
    
    # 각 그룹에 대해 엔트로피 계산
    def calculate_entropy(group):
        cell_counts = group['grid_cell'].value_counts()
        cell_probs = cell_counts / cell_counts.sum()
        return entropy(cell_probs, base=2)
    
    entropy_results = grouped.apply(calculate_entropy, include_groups=False).reset_index(name='day_location_entropy')
    
    return entropy_results

# val 데이터에 GPS 엔트로피 계산 및 원본 데이터에 추가
val_entropy = calculate_gps_entropy(val)
val = pd.merge(val, val_entropy, on=['subject_id', 'day'], how='left')

# test 데이터에 GPS 엔트로피 계산 및 원본 데이터에 추가
test_entropy = calculate_gps_entropy(test)
test = pd.merge(test, test_entropy, on=['subject_id', 'day'], how='left')

C:\Users\lty\AppData\Local\Temp\ipykernel_22172\3846522728.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gps.loc[:, 'grid_cell'] = df_gps.apply(lambda row: get_grid_cell(row['latitude_median'], row['longitude_median']), axis=1)
C:\Users\lty\AppData\Local\Temp\ipykernel_22172\3846522728.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gps.loc[:, 'grid_cell'] = df_gps.apply(lambda row: get_grid_cell(row['latitude_median'], row['longitude_median']), axis=1)


In [22]:
import numpy as np

# Haversine 공식을 이용한 두 지점 사이의 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반경 (킬로미터)
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c  # 결과는 킬로미터

# 하루 이동반경 계산 함수
def calculate_daily_range(group):
    if len(group) < 2:
        return 0  # 데이터가 2개 미만이면 이동반경을 0으로 설정
    
    lat_min, lat_max = group['latitude_median'].min(), group['latitude_median'].max()
    lon_min, lon_max = group['longitude_median'].min(), group['longitude_median'].max()
    range_distance = haversine(lat_min, lon_min, lat_max, lon_max)
    return range_distance

# val 데이터를 user와 day로 그룹화
grouped_val = val.groupby(['subject_id', 'day'])
grouped_test = val.groupby(['subject_id', 'day'])


# val 데이터에 하루 이동반경 계산 및 원본 데이터에 추가
val['day_Movement_radius'] = grouped_val.apply(calculate_daily_range).reset_index(drop=True)
test['day_Movement_radius'] = grouped_test.apply(calculate_daily_range).reset_index(drop=True)


C:\Users\lty\AppData\Local\Temp\ipykernel_22172\2844921695.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  val['day_Movement_radius'] = grouped_val.apply(calculate_daily_range).reset_index(drop=True)
C:\Users\lty\AppData\Local\Temp\ipykernel_22172\2844921695.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test['day_Movement_radius'] = grouped_test.apply(calculate_daily_range).reset_index(drop=True

In [23]:
import numpy as np

# heart 관련 열에 대해 0 값을 NaN으로 변경 (val 데이터)
heart_columns_val = [col for col in val.columns if 'heart' in col]
val[heart_columns_val] = val[heart_columns_val].replace(0, np.nan)

# heart 관련 열에 NaN이 있는 행의 특정 칼럼 값들도 NaN으로 변경 (val 데이터)
cols_to_update_val = ['w_light', 'burned_calories', 'distance', 'running_steps', 'speed', 'steps', 'step_frequency', 'walking_steps']
val.loc[val[heart_columns_val].isna().any(axis=1), cols_to_update_val] = val.loc[val[heart_columns_val].isna().any(axis=1), cols_to_update_val].replace(0, np.nan)

# heart 관련 열에 대해 0 값을 NaN으로 변경 (test 데이터)
heart_columns_test = [col for col in test.columns if 'heart' in col]
test[heart_columns_test] = test[heart_columns_test].replace(0, np.nan)

# heart 관련 열에 NaN이 있는 행의 특정 칼럼 값들도 NaN으로 변경 (test 데이터)
cols_to_update_test = ['w_light', 'burned_calories', 'distance', 'running_steps', 'speed', 'steps', 'step_frequency', 'walking_steps']
test.loc[test[heart_columns_test].isna().any(axis=1), cols_to_update_test] = test.loc[test[heart_columns_test].isna().any(axis=1), cols_to_update_test].replace(0, np.nan)


In [25]:
# HRV 계산
import neurokit2 as nk

def calculate_hrv_grouped(df, groupby=['user', 'day']):
    # Step 1: Grouping by user and day
    grouped = df.groupby(groupby)
    
    # Step 2: Initialize an empty DataFrame to store results
    final_df = pd.DataFrame()
    
    # Step 3: Process each group
    for group_name, group_data in grouped:
        # Step 4: Filter out invalid heart rate values
        group_data = group_data[group_data['heart_rate_median'] > 0]
        
        if len(group_data) < 2:
            print(f"Skipping group {group_name} due to insufficient data.")
            continue
        
        # Step 5: Convert HR data to RR intervals
        RR_intervals = 60000 / group_data['heart_rate_median']  # RR intervals in milliseconds
        
        # Step 6: Filter out invalid RR intervals
        RR_intervals = RR_intervals.replace([np.inf, -np.inf], np.nan).dropna()
        
        if len(RR_intervals) < 2:
            print(f"Skipping group {group_name} due to insufficient valid RR intervals.")
            continue
        
        # Step 7: Convert RR intervals to peaks
        peaks = nk.intervals_to_peaks(RR_intervals.values)
        
        # Step 8: Calculate HRV
        hrv_results = nk.hrv_time(peaks, sampling_rate=60) # Assume data is per minute
        
        # Step 9: Add results to a temporary DataFrame with user and day information
        hrv_df = pd.DataFrame(hrv_results, index=[0])
        hrv_df['subject_id'] = group_name[0]
        hrv_df['day'] = group_name[1]
        
        # Step 10: Concatenate to final_df
        final_df = pd.concat([final_df, hrv_df], ignore_index=True)
    
    return final_df

In [26]:
# 심박수 변이도를 구하기위한 데이터 별도 지정
hr_median_val = val[['subject_id','datetime','day','heart_rate_median']]
hr_median_test = test[['subject_id','datetime','day','heart_rate_median']]
# 함수 진행
hrv_results_val = calculate_hrv_grouped(hr_median_val, groupby=['subject_id', 'day'])
hrv_results_test = calculate_hrv_grouped(hr_median_test, groupby=['subject_id', 'day'])

# 원본 데이터에 병합하기
val = pd.merge(val , hrv_results_val, on=['subject_id','day'],how='left')
test = pd.merge(test , hrv_results_test, on=['subject_id','day'],how='left')

C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\neurokit2\hrv\hrv_time.py:232: RuntimeWarning: Mean of empty slice
  avg_rri.append(np.nanmean(rri[start_idx:end_idx]))
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\neurokit2\hrv\hrv_time.py:232: RuntimeWarning: Mean of empty slice
  avg_rri.append(np.nanmean(rri[start_idx:end_idx]))
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, d

Skipping group (1.0, Timestamp('2023-08-29 00:00:00')) due to insufficient data.


C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Skipping group (1.0, Timestamp('2023-09-12 00:00:00')) due to insufficient data.
Skipping group (1.0, Timestamp('2023-09-13 00:00:00')) due to insufficient data.
Skipping group (1.0, Timestamp('2023-09-19 00:00:00')) due to insufficient data.
Skipping group (1.0, Timestamp('2023-09-27 00:00:00')) due to insufficient data.


C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\neurokit2\hrv\hrv_time.py:232: RuntimeWarning: Mean of empty slice
  avg_rri.append(np.nanmean(rri[start_idx:end_idx]))
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis

...

C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  

Skipping group (8.0, Timestamp('2023-10-12 00:00:00')) due to insufficient data.


C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  

Skipping group (8.0, Timestamp('2023-10-21 00:00:00')) due to insufficient data.


C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\neurokit2\hrv\hrv_time.py:232: RuntimeWarning: Mean of empty slice
  avg_rri.append(np.nanmean(rri[start_idx:end_idx]))
C:\Users\lty\AppData\Local\anaconda3\envs\lee_ty\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis

In [27]:
val.columns

Index(['datetime', 'timestamp', 'subject_id', 'x_last', 'x_mean', 'x_max',
       'x_min', 'x_median', 'y_last', 'y_mean', 'y_max', 'y_min', 'y_median',
       'z_last', 'z_mean', 'z_max', 'z_min', 'z_median', 'm_activity',
       'ambience_labels', 'altitude_last', 'altitude_mean', 'altitude_max',
       'altitude_min', 'altitude_median', 'latitude_last', 'latitude_mean',
       'latitude_max', 'latitude_min', 'latitude_median', 'longitude_last',
       'longitude_mean', 'longitude_max', 'longitude_min', 'longitude_median',
       'speed_last', 'speed_mean', 'speed_max', 'speed_min', 'speed_median',
       'm_light', 'heart_rate_last', 'heart_rate_mean', 'heart_rate_max',
       'heart_rate_min', 'heart_rate_median', 'w_light', 'burned_calories',
       'distance', 'running_steps', 'speed', 'steps', 'step_frequency',
       'walking_steps', 'z_angle', 'ENMO', 'primary_ambience', 'sleep_sound',
       'day', 'day_location_entropy', 'day_Movement_radius', 'HRV_MeanNN',
       'HRV_SDNN'

In [28]:
val.drop(columns=['day'],inplace=True)
test.drop(columns=['day'],inplace=True)

In [29]:
# 'datetime' 컬럼을 datetime 형식으로 변환
val['datetime'] = pd.to_datetime(val['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

# 관심 있는 컬럼 선택 (ambience_labels 제외)
columns_of_interest = ['x_last', 'x_mean', 'x_max', 'x_min', 'x_median', 'y_last', 'y_mean', 'y_max', 'y_min', 'y_median',
                       'z_last', 'z_mean', 'z_max', 'z_min', 'z_median', 'speed_last', 'speed_mean', 'speed_max', 'speed_min', 'speed_median', 'm_light',
                       'heart_rate_last', 'heart_rate_mean', 'heart_rate_max', 'heart_rate_min', 'heart_rate_median',
                       'w_light', 'burned_calories', 'distance', 'running_steps', 'speed', 'steps', 'step_frequency', 'walking_steps']

# 최빈값을 계산할 컬럼
mode_columns = ['m_activity', 'altitude_last', 'altitude_mean', 'altitude_max', 'altitude_min', 'altitude_median',
                'latitude_last', 'latitude_mean', 'latitude_max', 'latitude_min', 'latitude_median', 'longitude_last',
                'longitude_mean', 'longitude_max', 'longitude_min', 'longitude_median',
                'day_location_entropy', 'day_Movement_radius', 'HRV_MeanNN',
                'HRV_SDNN', 'HRV_SDANN1', 'HRV_SDNNI1', 'HRV_SDANN2', 'HRV_SDNNI2',
                'HRV_SDANN5', 'HRV_SDNNI5', 'HRV_RMSSD', 'HRV_SDSD', 'HRV_CVNN',
                'HRV_CVSD', 'HRV_MedianNN', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN',
                'HRV_SDRMSSD', 'HRV_Prc20NN', 'HRV_Prc80NN', 'HRV_pNN50', 'HRV_pNN20',
                'HRV_MinNN', 'HRV_MaxNN', 'HRV_HTI', 'HRV_TINN']

# 결측치를 채우고 통계치를 계산하는 함수
def fill_missing_and_calculate(group):
    for column in columns_of_interest:
        if column in group:
            # 그룹이 전부 NaN인 경우를 방지
            if group[column].isna().all():
                if column in mode_columns:
                    group[column] = group[column].fillna(group[column].mode()[0] if not group[column].mode().empty else 0)
                else:
                    group[column] = 0
            else:
                if column in mode_columns:
                    group[column] = group[column].fillna(group[column].mode()[0] if not group[column].mode().empty else 0)
                else:
                    group[column] = group[column].fillna(group[column].median())
    stats = group[columns_of_interest].median().to_frame().T
    for column in mode_columns:
        if column in group:
            stats[column] = group[column].mode()[0] if not group[column].mode().empty else 0
    return stats

# 'datetime'을 인덱스로 설정
val.set_index('datetime', inplace=True)
test.set_index('datetime', inplace=True)

# subject_id와 datetime을 기준으로 데이터를 6시간 간격으로 그룹화하여 결측치 채우기 및 통계치 계산
val_stats = val.groupby(['subject_id', pd.Grouper(freq='24H', offset='0H')]).apply(fill_missing_and_calculate).reset_index()
test_stats = test.groupby(['subject_id', pd.Grouper(freq='24H', offset='0H')]).apply(fill_missing_and_calculate).reset_index()

# 결과 확인
print(val_stats.head())
print(test_stats.head())

C:\Users\lty\AppData\Local\Temp\ipykernel_22172\1428552264.py:48: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  val_stats = val.groupby(['subject_id', pd.Grouper(freq='24H', offset='0H')]).apply(fill_missing_and_calculate).reset_index()
C:\Users\lty\AppData\Local\Temp\ipykernel_22172\1428552264.py:48: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  val_stats = val.groupby(['subject_id', pd.Grouper(freq='24H', offset='0H')]).apply(fill_missing_and_calculate).reset_index()
C:\Users\lty\AppData\Local\Temp\ipykernel_22172\1428552264.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  val_

   subject_id   datetime  level_2    x_last    x_mean     x_max     x_min  \
0         1.0 2023-08-20        0  0.133998  0.219183  0.981057 -0.765703   
1         1.0 2023-08-21        0  0.129212  0.169542  1.319642 -1.676172   
2         1.0 2023-08-22        0  1.045664  1.056068  1.262214 -0.154337   
3         1.0 2023-08-23        0  0.795614  0.882982  6.075376 -0.905683   
4         1.0 2023-08-24        0 -0.062213 -0.032638  1.344766 -1.586441   

   x_median    y_last    y_mean  ...    HRV_IQRNN  HRV_SDRMSSD   HRV_Prc20NN  \
0  0.150748  0.413958  0.428777  ...  3033.333333     1.407370  11500.000000   
1  0.140578 -0.009571 -0.013229  ...  2233.333333     1.326863  10533.333333   
2  1.050449 -1.746760 -1.750896  ...  2333.333333     1.268400  11116.666667   
3  0.757328  0.032901  0.044782  ...  3033.333333     1.188516  10983.333333   
4 -0.033500 -0.111266 -0.112217  ...  2783.333333     1.273447  11350.000000   

    HRV_Prc80NN  HRV_pNN50  HRV_pNN20    HRV_MinNN     H

C:\Users\lty\AppData\Local\Temp\ipykernel_22172\1428552264.py:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_stats = test.groupby(['subject_id', pd.Grouper(freq='24H', offset='0H')]).apply(fill_missing_and_calculate).reset_index()


In [30]:
val_stats_path = 'val_24hr_median_0628.csv'
val_stats.to_csv(val_stats_path, index=False)
test_stats_path = 'test_24hr_median_0628.csv'
test_stats.to_csv(test_stats_path, index=False)
test_stats

,subject_id,datetime,level_2,x_last,x_mean,x_max,x_min,x_median,y_last,y_mean,...,HRV_IQRNN,HRV_SDRMSSD,HRV_Prc20NN,HRV_Prc80NN,HRV_pNN50,HRV_pNN20,HRV_MinNN,HRV_MaxNN,HRV_HTI,HRV_TINN
0,5.0,2023-11-05,0,-0.040701,-0.065710,1.574188,-1.702277,-0.064643,0.175974,0.184720,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,5.0,2023-11-06,0,0.028730,0.003530,1.690306,-0.938527,0.019752,0.414197,0.440236,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,5.0,2023-11-07,0,0.052672,0.038821,0.417788,-0.201113,0.041899,0.156820,0.168071,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,5.0,2023-11-08,0,-0.059855,-0.066900,0.238223,-0.715866,-0.069432,0.210690,0.217838,...,0.000000,0.000000,10000.000000,10000.000000,0.000000,0.000000,10000.000000,10000.000000,1.000000,0.0
4,5.0,2023-11-09,0,-0.390255,-0.376375,0.055067,-1.380257,-0.375890,0.294487,0.300855,...,16.666667,9.708690,12333.333333,12350.000000,0.208333,0.208333,10000.000000,12666.666667,1.716329,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,8.0,2023-11-05,0,0.126897,0.118722,0.265765,0.011971,0.119714,0.186753,0.185070,...,13358.333333,3.138471,11100.000000,25966.666667,93.667158,94.256259,8850.000000,31733.333333,42.437500,0.0
117,8.0,2023-11-06,0,-1.893871,-1.592407,0.011971,-8.382356,-1.468888,4.680807,4.672112,...,1600.000000,0.877626,10150.000000,12200.000000,91.330891,93.162393,8266.666667,28166.666667,35.608696,0.0
118,8.0,2023-11-07,0,0.189148,0.188317,0.284919,0.099362,0.189148,0.095771,0.101851,...,3241.666667,1.940019,10983.333333,19526.666667,82.622601,83.475480,5516.666667,30316.666667,39.083333,0.0
119,8.0,2023-11-08,0,0.385478,0.383590,0.823631,0.325621,0.385478,0.349564,0.350324,...,12400.000000,3.789745,10050.000000,24100.000000,83.501006,84.708249,7066.666667,30316.666667,45.181818,0.0
